### compare distribution of mutation effects on cell entry for F and RBP proteins

In [ ]:
import pandas as pd
import altair as alt
import httpimport
import numpy as np
import scipy.stats

_ = alt.data_transformers.disable_max_rows()

In [ ]:
# Import custom altair theme from remote github using httpimport module
def import_theme_new():
    with httpimport.github_repo("bblarsen-sci", "altair_themes", "main"):
        import main_theme

        @alt.theme.register("custom_theme", enable=True)
        def custom_theme():
            return main_theme.main_theme()


import_theme_new()

In [ ]:
F_df = pd.read_csv('../../../results/func_effects/averages/Nipah-F-CHO-bEFNB3_func_effects.csv').assign(protein='F')
RBP_df = pd.read_csv('../../../../../2023/Nipah_Malaysia_RBP_DMS/results/func_effects/averages/CHO_bEFNB3_func_effects.csv').assign(protein='RBP')

combined_df = pd.concat([F_df, RBP_df], ignore_index=True)
display(combined_df)

In [ ]:
#filter
filtered_df = combined_df[
    (combined_df['mutant'] != '*') &
    (combined_df['mutant'] != '-') &
    (combined_df['wildtype'] != '*') &
    (combined_df['times_seen'] >= 2) &
    (combined_df['effect_std'] <= 1) 
]

subset_filter_df = filtered_df[['site','wildtype','mutant','effect','protein']].round(2)
display(subset_filter_df)

In [ ]:
effect_distribution_chart = (
    alt.Chart(filtered_df)
    .mark_bar(opacity=0.5, binSpacing=1)
    .encode(
        x=alt.X(
            "effect:Q",
            title="Effect",
        ),
        y=alt.Y("count():Q", title="Count").stack(None),
        color=alt.Color(
            "protein:N",
            title="Protein",
        ),
    )
    .transform_bin("effect", field="effect", bin=alt.Bin(step=0.1))
)
display(effect_distribution_chart)


In [ ]:
density_chart = alt.Chart(filtered_df).transform_density(
        density='effect',
        groupby=['protein'],
        as_=['effect', 'density'],
        resolve='independent',
        extent=[-4,1],
    ).mark_area(opacity=0.6, stroke='black').encode(
        alt.X('effect', title='Effect'),
        alt.Y('density:Q', title='Density').stack(None),
        alt.Color('protein:N', scale=alt.Scale(domain=['F', 'RBP'], range=['#1f77b4', '#ff7f0e']))
)

display(density_chart)
#density_chart.save('../../results/figures/b2_b3_correlation/RBP_vs_F_effect_density.png',ppi=300)
#density_chart.save('../../results/figures/b2_b3_correlation/RBP_vs_F_effect_density.svg')